# Daily Stopwatch Data Science: KAThai Visualization

Note: this is a part of visualizing Khan Acadmy Thailand videos information. For more information, see https://khanacademythailand.org

## Stage 1: Ask a question

My objective is to create a visualization tool for Khan Academy Thailand videos.

There is no performance measure for the visualization yet.

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory.

**Note:** in order to work properly, you need to set up a terminal with UTF-8 format. See more [here](http://stackoverflow.com/questions/10561923/unicodedecodeerror-ascii-codec-cant-decode-byte-0xef-in-position-1).

In [1]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
#Set up the environment
import pandas as pd                        #Pandas
import numpy as np                         #Numpy

#Reading directly from Google sheets is complicated. Let's do a simple version by going to a local CSV file. 
#For more information about Google Sheets API, see https://developers.google.com/sheets/api/quickstart/python
'''
from StringIO import StringIO              #Properly read goodle sheets got moved to io in python3.

import requests

#read Google sheets data
r = requests.get('https://docs.google.com/spreadsheets/d/1uJSAtyUTQX9pmbI8YlXg_3Z15vKy3Bu4-wvzmoYOep0&output=csv')
data = r.content

df = pd.read_csv(StringIO(data))#, index_col=0)
'''

# Set up data directory
DataDir = "C:/Users/Admin/Documents/data/"
filename = "ThaiKAVideoProcess.xlsx"

# Set up output directory
OutputDir = "C:/Users/Admin/Documents/GitHub/KhanAcademyThailand/"

#Read data
df = pd.pandas.read_excel(DataDir+filename,sheetname='AllAvailableThaiVideos')

In [3]:
df[:10]

,Domain,DomainThai,Subject,SubjectThai,Topic,TopicThai,Tutorial,TutorialThai,ThaiCurriculumMapping,IsComplete,...,KASiteNameThai,KA site link,dub (Y/N)?,sub (Y/N)?,Original YoutubeID,Subbed Youtube Link,Dubbed Youtube Link,Dubbed Youtube ID,[YoutubeEnglishName] [[YoutubeThaiName]],Note
0,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting small numbers,การนับจำนวนน้อยๆ,ประถมต้น,1.0,...,การนับจำนวนน้อยๆ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,y2-uaPiyoxc,https://www.youtube.com/v/y2-uaPiyoxc&cc_load_...,https://www.youtube.com/v/eHNc9pnmHF8&cc_load_...,eHNc9pnmHF8,NaN,NaN
1,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting small numbers,การนับจำนวนน้อยๆ,ประถมต้น,1.0,...,การนับตามลำดับ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,PEeUTQ0Gri8,https://www.youtube.com/v/PEeUTQ0Gri8&cc_load_...,https://www.youtube.com/v/h6OTaBtdZ20&cc_load_...,h6OTaBtdZ20,NaN,NaN
2,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Numbers 0 to 120,จำนวน 0 ถึง 120,ประถมต้น,1.0,...,ตารางจำนวน,https://www.khanacademy.org/math/early-math/cc...,Y,Y,9XZypM2Z3Ro,https://www.youtube.com/v/9XZypM2Z3Ro&cc_load_...,https://www.youtube.com/v/WF1GsvP5_NU&cc_load_...,WF1GsvP5_NU,NaN,NaN
3,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Numbers 0 to 120,จำนวน 0 ถึง 120,ประถมต้น,1.0,...,จำนวนที่หายไประหว่าง 0 ถึง 120...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,1AqkBdCBm9o,https://www.youtube.com/v/1AqkBdCBm9o&cc_load_...,https://www.youtube.com/v/KppjuJsfn-U&cc_load_...,KppjuJsfn-U,NaN,NaN
4,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,ประถมต้น,1.0,...,การนับในภาพ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,leDYnoNSvD4,https://www.youtube.com/v/leDYnoNSvD4&cc_load_...,https://www.youtube.com/v/nSWEDZFHYGs&cc_load_...,nSWEDZFHYGs,NaN,NaN
5,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,ประถมต้น,1.0,...,การนับวาฬ แกะ และดอกไม้...,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,I9S5CvSqb5A,https://www.youtube.com/v/I9S5CvSqb5A&cc_load_...,NaN,NaN,NaN,NaN
6,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,ประถมต้น,1.0,...,การนับหมา หนู และคุกกี้...,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,EUqhLxFccbM,https://www.youtube.com/v/EUqhLxFccbM&cc_load_...,NaN,NaN,NaN,NaN
7,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,ประถมต้น,1.0,...,การเปรียบเทียบจำนวนสิ่งของ...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,__nkbr6DeTg,https://www.youtube.com/v/__nkbr6DeTg&cc_load_...,https://www.youtube.com/v/scTXzz6UQ0I&cc_load_...,scTXzz6UQ0I,NaN,NaN
8,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,ประถมต้น,1.0,...,การเปรียบเทียบจำนวนน้อยๆ บนเส้นจำนวน...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,tJrSILRXOUc,https://www.youtube.com/v/tJrSILRXOUc&cc_load_...,https://www.youtube.com/v/MnH9iXreDyU&cc_load_...,MnH9iXreDyU,NaN,NaN
9,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,ประถมต้น,1.0,...,นับตามประเภท,https://www.khanacademy.org/math/early-math/cc...,Y,Y,UA975j_qsTQ,https://www.youtube.com/v/UA975j_qsTQ&cc_load_...,https://www.youtube.com/v/gD1SlpnSRiI&cc_load_...,gD1SlpnSRiI,NaN,NaN


We will use an interactive visualization tool called Bokeh. See an [overview](http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/quickstart/quickstart.ipynb) and [how to install](http://bokeh.pydata.org/en/0.11.0/docs/installation.html).

In [4]:
from bokeh.models import ColumnDataSource, HoverTool, OpenURL, TapTool, Div
from bokeh.plotting import figure, show
from bokeh.sampledata.periodic_table import elements
from bokeh.io import vplot

from bokeh.layouts import row, widgetbox, layout
from bokeh.models import Select, TextInput
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg

from bokeh.embed import components

In [5]:
#make output appeared in notebook
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

## Stage 3: Explore the data

In [6]:
#Read description to patch in the output
desc = Div(text=open(DataDir +"description.html").read(), width=800)

In [7]:
d_complete = df[(df.IsComplete==1)|(df.IsComplete==0)]
d_complete.reset_index(inplace=True)

Here, we want to assign color based on the attributed called **ThaiCurriculumMapping**.

In [8]:
#DomainList = list(d_complete.Domain.unique())
DomainList = list(d_complete.DomainThai.unique())

In [9]:
d_complete['Color'] = "0"
for i in range(0,len(d_complete)):
#    for j in range(0,len(MappingList)):
    if d_complete.ThaiCurriculumMapping[i] == 'ประถมต้น':
        d_complete.loc[i,'Color'] = "#800080"#"purple",
    if d_complete.ThaiCurriculumMapping[i] == 'ประถมปลาย':
        d_complete.loc[i,'Color'] = "#0000FF"#"blue",
    if d_complete.ThaiCurriculumMapping[i] == 'มัธยมต้น':
        d_complete.loc[i,'Color'] = "#008000"#"green",
    if d_complete.ThaiCurriculumMapping[i] == 'มัธยมปลาย':
        d_complete.loc[i,'Color'] = "#FFFF00"#"yellow",
    if d_complete.ThaiCurriculumMapping[i] == 'มหาวิทยาลัย':
        d_complete.loc[i,'Color'] = "#FFA500"#"orange",
    if d_complete.ThaiCurriculumMapping[i] == 'ทั่วไป':
        d_complete.loc[i,'Color'] = "#FF0000"#"red",


Now we want to assign group number to each row. According to the order, we know that the rows under the same topic/subject are next to each other. So we will assign number using loop. 

In [10]:
d_complete['GroupNumber'] = 1
num = 1
for i in range(1,len(d_complete)):
    if (d_complete.Topic[i] == d_complete.Topic[i-1])&(d_complete.Subject[i] == d_complete.Subject[i-1]):
        num = num + 1
    else:
        num = 1
    d_complete.loc[i,'GroupNumber'] = num    
MaxGroupNumber = np.max(d_complete['GroupNumber'])

group_range = [str(x) for x in range(0, MaxGroupNumber + 1)] #This is equivalent to Group number in Periodic Table (columns)

Next, let's detect the smaller category called Subject.

In [11]:
# fill in Youtube link (redirect to translation page if needed)
for i in d_complete.index:
    if d_complete.loc[i,'IsComplete']==1:
        if d_complete.loc[i,'dub (Y/N)?'] == 'Y':
            d_complete.loc[i,'Subbed Youtube Link'] = d_complete.loc[i,'Dubbed Youtube Link'] 
    if d_complete.loc[i,'IsComplete']==0:
        d_complete.loc[i,'Subbed Youtube Link'] = "https://www.youtube.com/timedtext_video?v=" + d_complete.loc[i,'Original YoutubeID']
            

In [12]:
#Start HTML file
r_all = ''

In [13]:
DomainList = ["คณิตศาสตร์ตามวิชา","คณิตศาสตร์ตามระดับชั้น","วิทยาศาสตร์","คอมพิวเตอร์","ศิลปะและมนุษยศาสตร์","เศรษฐศาสตร์และการเงิน","เตรียมสอบ","หน่วยงานที่ร่วมมือ","(ไม่มีโดเมน)"]
for j in range(0,len(DomainList)):
    df = d_complete[d_complete.DomainThai==DomainList[j]]
    #df = d_complete[d_complete.DomainThai=="คณิตศาสตร์"]
    #df = d_complete[d_complete.DomainThai=="วิทยาศาสตร์"]
    #df = d_complete[d_complete.DomainThai=="คอมพิวเตอร์"]
    #df = d_complete[d_complete.DomainThai=="ศิลปะและมนุษยศาสตร์"]
    #df = d_complete[d_complete.DomainThai=="เศรษฐศาสตร์และการเงิน"]
    #df = d_complete[d_complete.DomainThai=="เตรียมสอบ"]
    #df = d_complete[d_complete.DomainThai=="หน่วยงานที่ร่วมมือ"]
    #df = d_complete[d_complete.DomainThai=="(ไม่มีโดเมน)"]
    df.reset_index(inplace=True)
    
    #SubjectList = list(df.Subject.unique())
    SubjectList = list(df.SubjectThai.unique())
    SubplotList = [0]*len(SubjectList)    #Create subplots for different subjects
    for i in range(0,len(SubjectList)):
        df_0 = df[(df['IsComplete']==0)&(df['SubjectThai']==SubjectList[i])]
        df_1 = df[(df['IsComplete']==1)&(df['SubjectThai']==SubjectList[i])]
        df_2 = df[df['GroupNumber']==1]
        df_2['GroupNumber'] = 0
        df_2['Tutorial'] = df_2['Topic']
        df_2['TutorialThai'] = df_2['TopicThai']
        df_2['KASiteName'] = df_2['Topic']
        df_2['KASiteNameThai'] = df_2['TopicThai']
        #Here we will associate each row with each topic. This is equivalent to Roman Letter in Periodic Table (rows)

        source0 = ColumnDataSource(
                data=dict(
                    GroupNumber= [str(x) for x in df_0['GroupNumber']],
                    #Topic = [str(x) for x in df_0['Topic']],
                    Topic = [str(x) for x in df_0['TopicThai']],
                    Tutorial = df_0['Tutorial'],
                    TutorialThai = df_0['TutorialThai'],
                    KASiteName = df_0['KASiteName'],
                    KASiteNameThai = df_0['KASiteNameThai'],
                    SiteLink = df_0['Subbed Youtube Link'],
                    #type_color= [colormap[x] for x in df_0['Color']],
                    type_color = df_0['Color'],
                    alpha = [0.7]*len(df_0),   
                )
            )
        source1 = ColumnDataSource(
                data=dict(
                    GroupNumber= [str(x) for x in df_1['GroupNumber']],
                    #Topic = [str(x) for x in df_1['Topic']],
                    Topic = [str(x) for x in df_1['TopicThai']],
                    Tutorial = df_1['Tutorial'],
                    TutorialThai = df_1['TutorialThai'],
                    KASiteName = df_1['KASiteName'],
                    KASiteNameThai = df_1['KASiteNameThai'],
                    SiteLink = df_1['Subbed Youtube Link'],
                    #type_color= [colormap[x] for x in df_1['Color']],
                    type_color = df_1['Color'],
                    alpha = [0.7]*len(df_1),   
                )
            )
        source2 = ColumnDataSource(
                data=dict(
                    GroupNumber= [str(x) for x in df_2['GroupNumber']],
                    Topic = [str(x) for x in df_2['TopicThai']],
                    Tutorial = df_2['Tutorial'],
                    TutorialThai = df_2['TutorialThai'],
                    KASiteName = df_2['KASiteName'],
                    KASiteNameThai = df_2['KASiteNameThai'],
                    SiteLink = df_2['KA site link'],
                    type_color = df_2['Color'],
                    alpha = [0.7]*len(df_2),   
                )
            )

        #TopicList = list(df[((df['IsComplete']==1)|(df['IsComplete']==0))&(df['Subject']==SubjectList[i])].Topic.unique()) 
        TopicList = list(df[((df['IsComplete']==1)|(df['IsComplete']==0))&(df['SubjectThai']==SubjectList[i])].TopicThai.unique()) 

        SubplotList[i] = figure(title= SubjectList[i], tools="tap,hover",
                   x_range=group_range, y_range=list(reversed(TopicList)))
        SubplotList[i].plot_width = 1800
        SubplotList[i].plot_height = 24*(len(TopicList)+2)
        SubplotList[i].toolbar_location = None
        SubplotList[i].outline_line_color = None

        SubplotList[i].circle("GroupNumber", "Topic", radius = 0.3, color='type_color', source=source0,
           fill_alpha="alpha")

        SubplotList[i].rect("GroupNumber", "Topic", 0.9, 0.9, color= 'type_color', source=source1,
            fill_alpha="alpha")

        SubplotList[i].asterisk("GroupNumber", "Topic", size = 9, source=source2)

        SubplotList[i].grid.grid_line_color = None

        #Hover to get information
        SubplotList[i].select_one(HoverTool).tooltips=[
                    ("Tutorial","@Tutorial"),
                    ("TutorialThai","@TutorialThai"),
                    ("KASiteName","@KASiteName"),
                    ("KASiteNameThai","@KASiteNameThai"),
                ]

        #Click to go to the website
        url = "@SiteLink"
        taptool = SubplotList[i].select(type=TapTool)
        taptool.callback = OpenURL(url=url)

        #SubplotList[i] = p
    # This is suppose to be combined plot    
    n = len(SubjectList)
    l = "q=vplot("
    c = ["SubplotList["+str(i)+"]," for i in range(0,n)]
    for i in range(0,n):
        l = l + c[i]
    l = l + ")"
    exec(l)
    
    script, div = components(q)
    script = '\n'.join([' ' + line for line in script.split('\n')])
    r = '''{script}
    
    <a name="figure"></a>
    {div}

    '''.format(script=script, div=div)
    r_head = '<div id="Domain' + str(j+1) +'" class="tabcontent"><h3>' + DomainList[j]+ '</h3>'
    r = r_head + r
    r = r +'</div>'
    r_all = r_all + r


In [14]:
text_file = open(DataDir+"body.html", "w")
text_file.write("%s" % r_all)
text_file.close()

#add this into the main HTML file

In [15]:
filenames = [DataDir+'head.html', DataDir+'body.html', DataDir+'tail.html']
with open(OutputDir+'index.html', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

Now the output file is ready-to-use HTML file. Not fully automated but not too hard to update by hand. Just reload the data file and rerun this code.

There are some ideas to develop this further:

1. Integrate seach capability
2. Do something to distinquish different tutorials under the same topic. Add a space!?
3. Make sure the font size is big enough.
4. Add more content from other website. Tool to scrap URL: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ 

*Note to myself:* I haven't tracked Pomodoros. I stuck for too long. Now it is up and running even though it is not the smartest solution at a moment.